# Data

In [ ]:
import string
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('data.csv')
df.head()

In [ ]:
classes = pd.value_counts(df["language"])
classes.plot(kind = 'pie',  autopct='%0.f%%') 
plt.title("Classes distribution")
plt.ylabel('')
plt.show()

In [ ]:
def RemovePun(text):
    for pun in string.punctuation:
        text = text.replace(pun, "")
    text = text.lower()
    return(text)
df["text"] = df["text"].apply(RemovePun)
df = df[['text', 'language']]
df.head()

#Model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import feature_extraction
from sklearn import pipeline
from sklearn import linear_model
from sklearn import metrics
import pickle

In [ ]:
X = df.iloc[:,0]
Y = df.iloc[:,1]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size= .2)
vectorizer = feature_extraction.text.TfidfVectorizer(ngram_range=(1,2), analyzer="char")
LanguagePipe = pipeline.Pipeline([("vec", vectorizer), ("clf", linear_model.LogisticRegression())])

In [ ]:
LanguagePipe.fit(X_train, Y_train)

Pipeline(steps=[('vec', TfidfVectorizer(analyzer='char', ngram_range=(1, 2))),
                ('clf', LogisticRegression())])

In [ ]:
PredictTest = LanguagePipe.predict(X_test)
metrics.accuracy_score(Y_test, PredictTest)*100

99.9730072049999

In [ ]:
metrics.confusion_matrix(Y_test, PredictTest)

array([[34600,     0],
       [   13, 13548]])

In [ ]:
print(LanguagePipe.predict(["Привет"]))
print(LanguagePipe.predict(["Hello"]))
print(LanguagePipe.predict(["Как дела"]))

array(['Russian'], dtype=object)

In [ ]:
NewFile = open("LanguageModel.pckl", "wb")
pickle.dump(LanguagePipe, NewFile)
NewFile.close()

#Web-test  


In [ ]:
pip install streamlit
import streamlit as st
import webbrowser

In [ ]:
global LanguageModel
LangModelFile = open('LanguageModel.pckl','rb')
LanguageModel = pickle.load(LangModelFile)
LangModelFile.close()
st.title("Language detection")
InputTest = st.text_input("Please, write your text here", "Hi, my name's Olga")
click = st.button("Get a language")
if click:
	st.text(LanguageModel.predict([InputTest]))
!npm install localtunnel
# cloud:
#streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com
# local:
#!streamlit run app.py &>/content/logs.txt & npx localtunnel